# Endpoint with Model Monitoring

In this Notebook we will deploy our model to a real time endpoint with model monitoring enabled.

We will hit the endpoint with some test traffic and then inspect the data that is been collected by the model monitoring service.

In [2]:
import os
import boto3
import re
import json
from sagemaker import get_execution_role, session

region= boto3.Session().region_name

role = get_execution_role()
print("RoleArn: {}".format(role))

# You can use a different bucket, but make sure the role you chose for this notebook
# has the s3:PutObject permissions. This is the bucket into which the data is captured
bucket =  session.Session(boto3.Session()).default_bucket()
print("Demo Bucket: {}".format(bucket))
prefix = 'Model-Monitor'

data_capture_prefix = '{}/datacapture'.format(prefix)
s3_capture_upload_path = 's3://{}/{}'.format(bucket, data_capture_prefix)
reports_prefix = '{}/reports'.format(prefix)
s3_report_path = 's3://{}/{}'.format(bucket,reports_prefix)
code_prefix = '{}/code'.format(prefix)
s3_code_preprocessor_uri = 's3://{}/{}/{}'.format(bucket,code_prefix, 'preprocessor.py')
s3_code_postprocessor_uri = 's3://{}/{}/{}'.format(bucket,code_prefix, 'postprocessor.py')

print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))
print("Preproc Code path: {}".format(s3_code_preprocessor_uri))
print("Postproc Code path: {}".format(s3_code_postprocessor_uri))

RoleArn: arn:aws:iam::320389841409:role/service-role/AmazonSageMaker-ExecutionRole-20201022T141998
Demo Bucket: sagemaker-ap-northeast-2-320389841409
Capture path: s3://sagemaker-ap-northeast-2-320389841409/Model-Monitor/datacapture
Report path: s3://sagemaker-ap-northeast-2-320389841409/Model-Monitor/reports
Preproc Code path: s3://sagemaker-ap-northeast-2-320389841409/Model-Monitor/code/preprocessor.py
Postproc Code path: s3://sagemaker-ap-northeast-2-320389841409/Model-Monitor/code/postprocessor.py


## Deploy the XGBoost Model

In [3]:
# This is our old friend the XGBoost model built earlier
# NOTE: Replace this URI with the path to the model that you build
model_url = 's3://sagemaker-ap-northeast-2-320389841409/sagemaker-xgboost-2021-06-16-12-41-25-643/output/model.tar.gz'


In [5]:
from time import gmtime, strftime
from sagemaker.model import Model
from sagemaker.image_uris import retrieve

model_name = "DEMO-xgb-churn-pred-model-monitor-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
image_uri = retrieve(region=boto3.Session().region_name, framework='xgboost', version="1.0-1")

model = Model(image_uri=image_uri, model_data=model_url, role=role)

In [4]:
#endpoint_name = 'DEMO-xgb-churn-pred-model-monitor-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_name = "DEMO-xgb-churn-pred-model-monitor-2021-06-22-05-05-36"

In [6]:
from sagemaker.model_monitor import DataCaptureConfig

print("EndpointName={}".format(endpoint_name))

data_capture_config = DataCaptureConfig(
                        enable_capture=True,
                        sampling_percentage=100,
                        destination_s3_uri=s3_capture_upload_path)

predictor = model.deploy(initial_instance_count=1,
                instance_type='ml.m4.xlarge',
                endpoint_name=endpoint_name,
                data_capture_config=data_capture_config)

EndpointName=DEMO-xgb-churn-pred-model-monitor-2021-06-22-05-05-36
-------------!

# Invoke the Endpoint

Run some test data through the endpoint

In [5]:
!head data/score.csv > data/sample.csv

In [6]:
!cat data/sample.csv

0,1,59.85,59.85,0,1,1,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,1,0
0,42,20.75,844.45,1,0,1,0,1,0,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,1,0,0,0
0,55,79.4,4238.45,0,1,0,1,0,1,0,1,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0
1,66,104.9,6891.45,1,0,0,1,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0
1,47,86.05,3865.6,0,1,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,1,0,0
0,17,56.1,946.95,0,1,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1
0,62,65.1,3846.75,1,0,1,0,1,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,0,1
0,46,19.25,864.2,1,0,0,1,0,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1
0,35,72.25,2568.55,1,0,0,1,0,1,0,1,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0
1,72,114.65,8333.95,1,0,0,1,1,0,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0


In [7]:
from sagemaker.predictor import Predictor
import sagemaker
import time

predictor = Predictor(endpoint_name=endpoint_name, serializer=sagemaker.serializers.CSVSerializer())

print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

with open('data/sample.csv', 'r') as f:
    for row in f:
        payload = row.rstrip('\n')
        response = predictor.predict(data=payload)
        time.sleep(0.5)
        
print("Done!")

Sending test traffic to the endpoint DEMO-xgb-churn-pred-model-monitor-2021-06-22-05-05-36. 
Please wait...
Done!


# View the Captured Data

We inspect the S3 location where all API call data has been logged.

You should expect to see different files from different time periods organized based on the hour in which the invocation occurred. 

The format of the Amazon S3 path is:

s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl


In [8]:
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
Model-Monitor/datacapture/DEMO-xgb-churn-pred-model-monitor-2021-06-22-05-05-36/AllTraffic/2021/06/23/04/51-42-261-172f445f-ca7b-4f8a-882d-31244eb444eb.jsonl
 Model-Monitor/datacapture/DEMO-xgb-churn-pred-model-monitor-2021-06-22-05-05-36/AllTraffic/2021/06/23/04/52-42-327-ed6e96af-99f7-4462-a01a-4957e29242e4.jsonl
 Model-Monitor/datacapture/DEMO-xgb-churn-pred-model-monitor-2021-06-22-05-05-36/AllTraffic/2021/06/23/04/53-42-730-3091ccd8-b80f-4f0b-ba13-54ac5af78d83.jsonl
 Model-Monitor/datacapture/DEMO-xgb-churn-pred-model-monitor-2021-06-22-05-05-36/AllTraffic/2021/06/23/04/54-43-146-89d531f8-ebf5-4100-9a69-2d476ee885b9.jsonl
 Model-Monitor/datacapture/DEMO-xgb-churn-pred-model-monitor-2021-06-22-05-05-36/AllTraffic/2021/06/23/04/55-43-598-c173f01d-038f-47a9-bd8c-edd51ee1eaa0.jsonl
 Model-Monitor/datacapture/DEMO-xgb-churn-pred-model-monitor-2021-06-22-05-05-36/AllTraffic/2021/06/23/04/56-44-030-28bf1322-fac2-4d4f-a74f-7b8b6a912aa4.jsonl
 Model-Monitor/datacapture

### Look Inside

Next, view the contents of a single capture file. Here you should see all the data captured in an Amazon SageMaker specific JSON-line formatted file. Take a quick peek at the first few lines in the captured file.

In [9]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])


{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"1,56,85.65,4824.45,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.14099116623401642","encoding":"CSV"}},"eventMetadata":{"eventId":"2068149a-f7a3-4fc2-ad0f-6aa360ac2e8f","inferenceTime":"2021-06-23T05:02:46Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"0,1,50.15,50.15,0,1,1,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0","encoding":"CSV"},"endpointOutput":{"observedContentType":"text/csv; charset=utf-8","mode":"OUTPUT","data":"0.07185926288366318","encoding":"CSV"}},"eventMetadata":{"eventId":"a454d31d-db4f-4863-ab45-8ddc368689c0","inferenceTime":"2021-06-23T05:02:46Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":

Look at an individual record 

In [10]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "1,56,85.65,4824.45,0,1,0,1,1,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0",
      "encoding": "CSV"
    },
    "endpointOutput": {
      "observedContentType": "text/csv; charset=utf-8",
      "mode": "OUTPUT",
      "data": "0.14099116623401642",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "2068149a-f7a3-4fc2-ad0f-6aa360ac2e8f",
    "inferenceTime": "2021-06-23T05:02:46Z"
  },
  "eventVersion": "0"
}



# PART B: Model Monitor - Baseling and continuous monitoring

In addition to collecting the data, Amazon SageMaker provides the capability for you to monitor and evaluate the data observed by the endpoints. For this:

* Create a baseline with which you compare the realtime traffic.
* Once a baseline is ready, setup a schedule to continously evaluate and compare against the baseline.




## 1. Constraint suggestion with baseline/training dataset

The training dataset with which you trained the model is usually a good baseline dataset. Note that the training dataset data schema and the inference dataset schema should exactly match (i.e. the number and order of the features).

From the training dataset you can ask Amazon SageMaker to suggest a set of baseline constraints and generate descriptive statistics to explore the data. For this example, upload the training dataset that was used to train the pre-trained model included in this example. If you already have it in Amazon S3, you can directly point to it.


In [11]:
baseline_data_uri = "s3://telco-churn-seoul/xgboost-example/train.csv"
baseline_prefix = prefix + '/baseline'
baseline_results_uri = 's3://{}/{}'.format(bucket, baseline_prefix)
print('Baseline data uri: {}'.format(baseline_data_uri))
print('Baseline results uri: {}'.format(baseline_results_uri))

Baseline data uri: s3://telco-churn-seoul/xgboost-example/train.csv
Baseline results uri: s3://sagemaker-ap-northeast-2-320389841409/Model-Monitor/baseline



## Create a baselining job with training dataset

Now that you have the training data ready in Amazon S3, start a job to suggest constraints. DefaultModelMonitor.suggest_baseline(..) starts a ProcessingJob using an Amazon SageMaker provided Model Monitor container to generate the constraints.


### Note: This next cell will take some time to execute

In [ ]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_default_monitor_baseline = my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)

### Above Cell output was cleared for readability of this Notebooks

# Explore the generated constraints and statistics

In [15]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_prefix)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Files:")
print("\n ".join(report_files))

Found Files:
Model-Monitor/baseline/constraints.json
 Model-Monitor/baseline/statistics.json


In [16]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,Senior Citizen,Integral,3899,0,0.162349,633.00,0.368771,0.00,1.00,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0,..."
1,Tenure,Integral,3899,0,32.352911,126144.00,24.460951,0.00,72.00,"[{'lower_bound': 0.0, 'upper_bound': 7.2, 'cou...",0.64,2048.0,"[[32.0, 16.0, 18.0, 69.0, 68.0, 1.0, 58.0, 55...."
2,Monthly Charges,Fractional,3899,0,64.958002,253271.25,30.091928,18.75,118.65,"[{'lower_bound': 18.75, 'upper_bound': 28.7400...",0.64,2048.0,"[[84.15, 71.8, 80.55, 66.9, 53.0, 69.75, 50.3,..."
3,Total Charges,Fractional,3895,4,2286.391130,8905493.45,2263.819521,18.80,8594.40,"[{'lower_bound': 18.8, 'upper_bound': 876.36, ...",0.64,2048.0,"[[3656.25, 69.75, 2878.55, 1001.5, 5125.75, 53..."
4,Churn,Integral,3899,0,0.263914,1029.00,0.440753,0.00,1.00,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,..."
5,Gender_Female,Integral,3899,0,0.488843,1906.00,0.499876,0.00,1.00,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
6,Gender_Male,Integral,3899,0,0.511157,1993.00,0.499876,0.00,1.00,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0,..."
7,Partner_No,Integral,3899,0,0.526289,2052.00,0.499308,0.00,1.00,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0,..."
8,Partner_Yes,Integral,3899,0,0.473711,1847.00,0.499308,0.00,1.00,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0,..."
9,Dependents_No,Integral,3899,0,0.706078,2753.00,0.455556,0.00,1.00,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0,..."


In [17]:
constraints_df = pd.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

,name,inferred_type,completeness,num_constraints.is_non_negative
0,Senior Citizen,Integral,1.000000,True
1,Tenure,Integral,1.000000,True
2,Monthly Charges,Fractional,1.000000,True
3,Total Charges,Fractional,0.998974,True
4,Churn,Integral,1.000000,True
5,Gender_Female,Integral,1.000000,True
6,Gender_Male,Integral,1.000000,True
7,Partner_No,Integral,1.000000,True
8,Partner_Yes,Integral,1.000000,True
9,Dependents_No,Integral,1.000000,True



### 2. Analyzing collected data for data quality issues

When you have collected the data above, analyze and monitor the data with Monitoring Schedules


In [18]:
# First, copy over some test scripts to the S3 bucket so that they can be used for pre and post processing
boto3.Session().resource('s3').Bucket(bucket).Object(code_prefix+"/preprocessor.py").upload_file('monitor/preprocessor.py')
boto3.Session().resource('s3').Bucket(bucket).Object(code_prefix+"/postprocessor.py").upload_file('monitor/postprocessor.py')

## Scheduled Monitoring

You can create a model monitoring schedule for the endpoint created earlier. Use the baseline resources (constraints and statistics) to compare against the realtime traffic.

In [19]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

mon_schedule_name = 'DEMO-xgb-churn-pred-model-monitor-schedule-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint_name,
    #record_preprocessor_script=pre_processor_script,
    post_analytics_processor_script=s3_code_postprocessor_uri,
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)



### Start generating some artificial traffic

The cell below starts a thread to send some traffic to the endpoint. Note that you need to stop the kernel to terminate this thread. If there is no traffic, the monitoring jobs are marked as Failed since there is no data to process.


In [21]:
from threading import Thread
from time import sleep
import time

endpoint_name=predictor.endpoint_name
runtime_client = boto3.client('runtime.sagemaker')

# (just repeating code from above for convenience/ able to run this section independently)
def invoke_endpoint(ep_name, file_name, runtime_client):
    with open(file_name, 'r') as f:
        for row in f:
            payload = row.rstrip('\n')
            response = runtime_client.invoke_endpoint(EndpointName=ep_name,
                                          ContentType='text/csv', 
                                          Body=payload)
            response['Body'].read()
            time.sleep(1)
            
def invoke_endpoint_forever():
    while True:
        invoke_endpoint(endpoint_name, 'data/score.csv', runtime_client)
        
thread = Thread(target = invoke_endpoint_forever)
thread.start()

# Note that you need to stop the kernel to stop the invocations


### Describe and inspect the schedule

Once you describe, observe that the MonitoringScheduleStatus changes to Scheduled.


In [22]:

desc_schedule_result = my_default_monitor.describe_schedule()
print('Schedule status: {}'.format(desc_schedule_result['MonitoringScheduleStatus']))


Schedule status: Scheduled



### List executions

The schedule starts jobs at the previously specified intervals. Here, you list the latest five executions. Note that if you are kicking this off after creating the hourly schedule, you might find the executions empty. You might have to wait until you cross the hour boundary (in UTC) to see executions kick off. The code below has the logic for waiting.

Note: Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule your execution. You might see your execution start in anywhere from zero to ~20 minutes from the hour boundary. This is expected and done for load balancing in the backend.


In [23]:
mon_executions = my_default_monitor.list_executions()
print("We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.\nWe will have to wait till we hit the hour...")

while len(mon_executions) == 0:
    print("Waiting for the 1st execution to happen...")
    time.sleep(60)
    mon_executions = my_default_monitor.list_executions()



No executions found for schedule. monitoring_schedule_name: DEMO-xgb-churn-pred-model-monitor-schedule-2021-06-23-23-31-10
We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.
We will have to wait till we hit the hour...
Waiting for the 1st execution to happen...
No executions found for schedule. monitoring_schedule_name: DEMO-xgb-churn-pred-model-monitor-schedule-2021-06-23-23-31-10
Waiting for the 1st execution to happen...
No executions found for schedule. monitoring_schedule_name: DEMO-xgb-churn-pred-model-monitor-schedule-2021-06-23-23-31-10
Waiting for the 1st execution to happen...
No executions found for schedule. monitoring_schedule_name: DEMO-xgb-churn-pred-model-monitor-schedule-2021-06-23-23-31-10
Waiting for the 1st execution to happen...
No executions found for schedule. monitoring_schedule_name: DEMO-xgb-churn-pred-model-monitor-schedule-2021-06-23-23-31-10
Waiting for the 1st execution to happen...
No executions found f


### Inspect a specific execution (latest execution)

In the previous cell, you picked up the latest completed or failed scheduled execution. Here are the possible terminal states and what each of them mean:

  * Completed - This means the monitoring execution completed and no issues were found in the violations report.
  * CompletedWithViolations - This means the execution completed, but constraint violations were detected.
  * Failed - The monitoring execution failed, maybe due to client error (perhaps incorrect role premissions) or infrastructure issues. Further examination of FailureReason and ExitMessage is necessary to identify what exactly happened.
  * Stopped - job exceeded max runtime or was manually stopped.



In [24]:
latest_execution = mon_executions[-1] # latest execution's index is -1, second to last is -2 and so on..
time.sleep(60)
latest_execution.wait(logs=False)

print("Latest execution status: {}".format(latest_execution.describe()['ProcessingJobStatus']))
print("Latest execution result: {}".format(latest_execution.describe()['ExitMessage']))

latest_job = latest_execution.describe()
if (latest_job['ProcessingJobStatus'] != 'Completed'):
        print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")

......................................................!Latest execution status: Completed
Latest execution result: CompletedWithViolations: Job completed successfully with 5 violations.


In [25]:
report_uri=latest_execution.output.destination
print('Report Uri: {}'.format(report_uri))


Report Uri: s3://sagemaker-ap-northeast-2-320389841409/Model-Monitor/reports/DEMO-xgb-churn-pred-model-monitor-2021-06-22-05-05-36/DEMO-xgb-churn-pred-model-monitor-schedule-2021-06-23-23-31-10/2021/06/24/00


### List the generated reports

In [26]:

from urllib.parse import urlparse
s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip('/')
print('Report bucket: {}'.format(report_bucket))
print('Report key: {}'.format(report_key))

s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Report Files:")
print("\n ".join(report_files))


Report bucket: sagemaker-ap-northeast-2-320389841409
Report key: Model-Monitor/reports/DEMO-xgb-churn-pred-model-monitor-2021-06-22-05-05-36/DEMO-xgb-churn-pred-model-monitor-schedule-2021-06-23-23-31-10/2021/06/24/00
Found Report Files:
Model-Monitor/reports/DEMO-xgb-churn-pred-model-monitor-2021-06-22-05-05-36/DEMO-xgb-churn-pred-model-monitor-schedule-2021-06-23-23-31-10/2021/06/24/00/constraint_violations.json
 Model-Monitor/reports/DEMO-xgb-churn-pred-model-monitor-2021-06-22-05-05-36/DEMO-xgb-churn-pred-model-monitor-schedule-2021-06-23-23-31-10/2021/06/24/00/constraints.json
 Model-Monitor/reports/DEMO-xgb-churn-pred-model-monitor-2021-06-22-05-05-36/DEMO-xgb-churn-pred-model-monitor-schedule-2021-06-23-23-31-10/2021/06/24/00/statistics.json



## Violations report

If there are any violations compared to the baseline, they will be listed here.


In [27]:
violations = my_default_monitor.latest_monitoring_constraint_violations()
pd.set_option('display.max_colwidth', None)
constraints_df = pd.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)



,feature_name,constraint_check_type,description
0,Churn,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
1,Senior Citizen,data_type_check,"Data type match requirement is not met. Expected data type: Integral, Expected match: 100.0%. Observed: Only 0.0% of data is Integral."
2,Monthly Charges,data_type_check,"Data type match requirement is not met. Expected data type: Fractional, Expected match: 100.0%. Observed: Only 0.0% of data is Fractional."
3,Churn,completeness_check,Data completeness requirement is not met. Expected: 100.0%. Observed: Only 99.85875706214689%.
4,Total Charges,baseline_drift_check,Baseline drift distance: 0.8334023561354106 exceeds threshold: 0.1



## Other commands

We can also start and stop the monitoring schedules.


In [28]:
#my_default_monitor.stop_monitoring_schedule()
#my_default_monitor.start_monitoring_schedule()


# Delete the resources

You can keep your endpoint running to continue capturing data. If you do not plan to collect more data or use this endpoint further, you should delete the endpoint to avoid incurring additional charges. Note that deleting your endpoint does not delete the data that was captured during the model invocations. That data persists in Amazon S3 until you delete it yourself.

But before that, you need to delete the schedule first.


In [29]:
my_default_monitor.delete_monitoring_schedule()
time.sleep(60) # actually wait for the deletion



Deleting Monitoring Schedule with name: DEMO-xgb-churn-pred-model-monitor-schedule-2021-06-23-23-31-10


In [30]:
predictor.delete_endpoint()


### Remeber to restart the kernel to stop the data being sent to the endpoint